In [1]:
import pandas as pd

df = pd.read_csv('data-p3/joined_all.csv')
df.head()


,Pass_ID,Sch_Departure,Act_Departure,C0 - S2,C_Start,C_Avg,S2,Wait_Time,Day_of_Week,Period_of_Week,Airfield,Season
0,5368296,2028-09-03 13:12,2028-09-03 13:12,1,1,1.0,2028-09-03 11:26,3.0,7 - SUN,2 - WEEKEND,AUC,3 - SUMMER
1,5480057,2028-09-15 9:36,2028-09-15 9:36,1,1,1.0,2028-09-15 7:45,3.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
2,5349077,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 7:58,5.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
3,5349207,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 8:04,4.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER
4,5349308,2028-09-01 10:21,2028-09-01 11:09,1,1,1.0,2028-09-01 8:13,7.0,5 - FRI,1 - WEEKDAY,AUC,3 - SUMMER


In [2]:
df.columns

Index(['Pass_ID', 'Sch_Departure', 'Act_Departure', 'C0 - S2', 'C_Start',
       'C_Avg', 'S2', 'Wait_Time', 'Day_of_Week', 'Period_of_Week', 'Airfield',
       'Season'],
      dtype='object')

In [3]:
df['S2'] = pd.to_datetime(df['S2'])
df['Wait_Time'] = pd.to_timedelta(df['Wait_Time'], unit='m')
df['S1'] =df['S2'] - df['Wait_Time']

In [4]:
cluster_ord = [
    "Weekday - 0:00 - 4:00", "Weekday - 4:00 - 8:00", "Weekday - 8:00 - 12:00", 
    "Weekday - 12:00 - 16:00", "Weekday - 16:00 - 20:00", "Weekday - 20:00 - 0:00",
    "Weekend - 0:00 - 4:00", "Weekend - 4:00 - 8:00", "Weekend - 8:00 - 12:00", 
    "Weekend - 12:00 - 16:00", "Weekend - 16:00 - 20:00", "Weekend - 20:00 - 0:00"
]

In [5]:
def determine_cluster(timestamp):

    is_weekend = timestamp.weekday() >= 5  
    hour = timestamp.hour
    
    if 0 <= hour < 4:
        cluster = "0:00 - 4:00"
    elif 4 <= hour < 8:
        cluster = "4:00 - 8:00"
    elif 8 <= hour < 12:
        cluster = "8:00 - 12:00"
    elif 12 <= hour < 16:
        cluster = "12:00 - 16:00"
    elif 16 <= hour < 20:
        cluster = "16:00 - 20:00"
    else:
        cluster = "20:00 - 0:00"
    
    return f"{'Weekend' if is_weekend else 'Weekday'} - {cluster}"

df['cluster'] = df['S2'].apply(determine_cluster)

In [6]:
df['cluster'] = pd.Categorical(df['cluster'], categories=cluster_ord, ordered=True)
df = df.sort_values('cluster')

In [7]:
df['cluster'].isna().any()

False

In [8]:
def cluster_hours(timestamp):
    is_weekend = timestamp.weekday() >= 5

    weekday = (153 - 44) * 4
    weekend = 44 * 4

    
    return weekend if is_weekend else weekday

df['cluster_hours'] = df['S2'].apply(cluster_hours)

In [9]:
df[['cluster','cluster_hours']]

,cluster,cluster_hours
57071,Weekday - 4:00 - 8:00,436
33473,Weekday - 4:00 - 8:00,436
33472,Weekday - 4:00 - 8:00,436
33471,Weekday - 4:00 - 8:00,436
33470,Weekday - 4:00 - 8:00,436
...,...,...
91986,Weekend - 20:00 - 0:00,176
66543,Weekend - 20:00 - 0:00,176
47906,Weekend - 20:00 - 0:00,176
77715,Weekend - 20:00 - 0:00,176


In [10]:
arrival_counts = df.groupby('cluster').size()
arrival_counts.head()


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_27366/253566319.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  arrival_counts = df.groupby('cluster').size()


cluster
Weekday - 0:00 - 4:00          0
Weekday - 4:00 - 8:00      15203
Weekday - 8:00 - 12:00     20785
Weekday - 12:00 - 16:00    21850
Weekday - 16:00 - 20:00    20992
dtype: int64

In [11]:
results = arrival_counts.to_frame(name='count')

results = results.merge(df[['cluster', 'cluster_hours']].drop_duplicates(), on='cluster', how='left')
results.head()


,cluster,count,cluster_hours
0,Weekday - 0:00 - 4:00,0,NaN
1,Weekday - 4:00 - 8:00,15203,436.0
2,Weekday - 8:00 - 12:00,20785,436.0
3,Weekday - 12:00 - 16:00,21850,436.0
4,Weekday - 16:00 - 20:00,20992,436.0


In [12]:
results['avg_arrival_rate'] = results['count'] / results['cluster_hours']
results


,cluster,count,cluster_hours,avg_arrival_rate
0,Weekday - 0:00 - 4:00,0,NaN,NaN
1,Weekday - 4:00 - 8:00,15203,436.0,34.869266
2,Weekday - 8:00 - 12:00,20785,436.0,47.672018
3,Weekday - 12:00 - 16:00,21850,436.0,50.114679
4,Weekday - 16:00 - 20:00,20992,436.0,48.146789
5,Weekday - 20:00 - 0:00,1546,436.0,3.545872
6,Weekend - 0:00 - 4:00,0,NaN,NaN
7,Weekend - 4:00 - 8:00,5973,176.0,33.937500
8,Weekend - 8:00 - 12:00,7472,176.0,42.454545
9,Weekend - 12:00 - 16:00,9494,176.0,53.943182


In [14]:
average_C0 = df.groupby('cluster')['C0 - S2'].mean().reset_index(name='avg_C0')
average_C0

/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_27366/441160710.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_C0 = df.groupby('cluster')['C0 - S2'].mean().reset_index(name='avg_C0')


,cluster,avg_C0
0,Weekday - 0:00 - 4:00,NaN
1,Weekday - 4:00 - 8:00,1.830626
2,Weekday - 8:00 - 12:00,1.264037
3,Weekday - 12:00 - 16:00,1.235103
4,Weekday - 16:00 - 20:00,1.372237
5,Weekday - 20:00 - 0:00,1.021992
6,Weekend - 0:00 - 4:00,NaN
7,Weekend - 4:00 - 8:00,1.776996
8,Weekend - 8:00 - 12:00,1.154042
9,Weekend - 12:00 - 16:00,1.203076


In [15]:
distribution_C0 = pd.crosstab(df['cluster'], df['C0 - S2'], normalize='index') * 100
distribution_C0.columns = [f'{col}_total_servers' for col in distribution_C0.columns]
distribution_C0


,0_total_servers,1_total_servers,2_total_servers,3_total_servers
cluster,,,,
Weekday - 4:00 - 8:00,0.000000,20.384135,76.169177,3.446688
Weekday - 8:00 - 12:00,0.000000,74.577821,24.440702,0.981477
Weekday - 12:00 - 16:00,0.000000,77.066362,22.356979,0.576659
Weekday - 16:00 - 20:00,0.000000,63.933880,34.908537,1.157584
Weekday - 20:00 - 0:00,0.064683,97.671410,2.263907,0.000000
Weekend - 4:00 - 8:00,0.000000,24.962330,72.375691,2.661979
Weekend - 8:00 - 12:00,0.000000,84.595824,15.404176,0.000000
Weekend - 12:00 - 16:00,0.000000,79.692437,20.307563,0.000000
Weekend - 16:00 - 20:00,0.000000,57.371104,38.698294,3.930602


In [16]:
results = average_C0.merge(distribution_C0, on='cluster', how='left')
results

,cluster,avg_C0,0_total_servers,1_total_servers,2_total_servers,3_total_servers
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,1.830626,0.000000,20.384135,76.169177,3.446688
2,Weekday - 8:00 - 12:00,1.264037,0.000000,74.577821,24.440702,0.981477
3,Weekday - 12:00 - 16:00,1.235103,0.000000,77.066362,22.356979,0.576659
4,Weekday - 16:00 - 20:00,1.372237,0.000000,63.933880,34.908537,1.157584
5,Weekday - 20:00 - 0:00,1.021992,0.064683,97.671410,2.263907,0.000000
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,1.776996,0.000000,24.962330,72.375691,2.661979
8,Weekend - 8:00 - 12:00,1.154042,0.000000,84.595824,15.404176,0.000000
9,Weekend - 12:00 - 16:00,1.203076,0.000000,79.692437,20.307563,0.000000


In [17]:
def imputation_cluster(timestamp):

    day_name = timestamp.strftime('%A')  # day of week
    hour = timestamp.hour
    
    if 0 <= hour < 4:
        time_cluster = "0:00 - 4:00"
    elif 4 <= hour < 8:
        time_cluster = "4:00 - 8:00"
    elif 8 <= hour < 12:
        time_cluster = "8:00 - 12:00"
    elif 12 <= hour < 16:
        time_cluster = "12:00 - 16:00"
    elif 16 <= hour < 20:
        time_cluster = "16:00 - 20:00"
    else:
        time_cluster = "20:00 - 0:00"
    

    return f"{day_name} - {time_cluster}"

df['imputation_cluster'] = df['S2'].apply(imputation_cluster)


In [18]:
imp_avg_wait = df.groupby('imputation_cluster')['Wait_Time'].mean()

df['wait_time_imp'] = df['Wait_Time'].fillna(df['imputation_cluster'].map(imp_avg_wait))
df['wait_time_imp'].hasnans


False

In [19]:
df['wait_time_imp'] = df['wait_time_imp'] / pd.Timedelta(minutes=1)

thresholds = [5, 10, 15, 20, 25, 30]

results = {}

for cluster, group in df.groupby('cluster'):
    results[cluster] = {f"<= {threshold} mins": (group['wait_time_imp'] <= threshold).mean() * 100
                        for threshold in thresholds}

percentage_table_by_cluster = pd.DataFrame(results).transpose()
percentage_table_by_cluster.index.name = 'cluster'
percentage_table_by_cluster.reset_index(inplace=True)
percentage_table_by_cluster


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_27366/112405527.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for cluster, group in df.groupby('cluster'):


,cluster,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN,NaN
1,Weekday - 4:00 - 8:00,40.373610,74.347168,90.153259,96.704598,99.046241,99.671118
2,Weekday - 8:00 - 12:00,58.768343,85.484725,93.524176,97.329805,98.720231,99.629541
3,Weekday - 12:00 - 16:00,72.311213,92.713959,97.748284,99.322654,99.615561,99.835240
4,Weekday - 16:00 - 20:00,78.791921,96.084223,99.047256,99.733232,99.919017,99.966654
5,Weekday - 20:00 - 0:00,96.895213,100.000000,100.000000,100.000000,100.000000,100.000000
6,Weekend - 0:00 - 4:00,NaN,NaN,NaN,NaN,NaN,NaN
7,Weekend - 4:00 - 8:00,49.857693,88.581952,97.907249,99.648418,99.949774,99.949774
8,Weekend - 8:00 - 12:00,67.277837,92.036938,97.430407,99.156852,99.692184,99.973233
9,Weekend - 12:00 - 16:00,74.257426,92.900779,98.714978,99.873604,99.989467,99.989467


In [20]:
df["S1_imp"] = df['S1'].fillna(df['S2'] - df['Wait_Time'])

In [21]:
import numpy as np

In [23]:
mins_per_cluster = 4 * 60

df['wait_time_minutes'] = df['Wait_Time'].dt.total_seconds() / 60

cluster_metrics = df.groupby('cluster').apply(
    lambda group: pd.Series({
        'arrival_rate': len(group) / mins_per_cluster,
        'average_wait_time': group['wait_time_minutes'].mean()
    })
).reset_index()

cluster_metrics['estimated_service_rate'] = (
    (cluster_metrics['average_wait_time'] * cluster_metrics['arrival_rate'] + 
     np.sqrt((cluster_metrics['average_wait_time'] * cluster_metrics['arrival_rate'])**2 + 
             4 * cluster_metrics['average_wait_time'] * cluster_metrics['arrival_rate'])) 
    / (2 * cluster_metrics['average_wait_time'])
)

cluster_metrics['traffic_intensity'] = cluster_metrics['arrival_rate'] / cluster_metrics['estimated_service_rate']

qos_results = []
for _, row in cluster_metrics.iterrows():
    cluster = row['cluster']
    arrival_rate = row['arrival_rate']
    estimated_service_rate = row['estimated_service_rate']
    
    qos_estimates = {}
    for x in thresholds:
        if estimated_service_rate > arrival_rate:
            qos = 1 - (arrival_rate / estimated_service_rate) * np.exp(-(estimated_service_rate - arrival_rate) * x)
            qos_estimates[f"<= {x} mins"] = f"{qos:.1%}"
        else:
            qos_estimates[f"<= {x} mins"] = "N/A"
    
    result = {
        "Cluster": cluster,
        "Est Serv Rate": round(row['estimated_service_rate'], 3),
        "Est ρ": round(row['traffic_intensity'], 3),
        **qos_estimates
    }
    qos_results.append(result)

qos_df = pd.DataFrame(qos_results)
qos_df


/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_27366/1411453126.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cluster_metrics = df.groupby('cluster').apply(
/var/folders/8t/19klq0xd5w7cf39d3893ts0w0000gn/T/ipykernel_27366/1411453126.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_metrics = df.groupby('cluster').apply(


,Cluster,Est Serv Rate,Est ρ,<= 5 mins,<= 10 mins,<= 15 mins,<= 20 mins,<= 25 mins,<= 30 mins
0,Weekday - 0:00 - 4:00,NaN,NaN,N/A,N/A,N/A,N/A,N/A,N/A
1,Weekday - 4:00 - 8:00,63.472,0.998,46.8%,71.7%,84.9%,92.0%,95.7%,97.7%
2,Weekday - 8:00 - 12:00,86.763,0.998,54.9%,79.7%,90.8%,95.9%,98.1%,99.2%
3,Weekday - 12:00 - 16:00,91.246,0.998,64.1%,87.1%,95.3%,98.3%,99.4%,99.8%
4,Weekday - 16:00 - 20:00,87.709,0.997,70.2%,91.1%,97.4%,99.2%,99.8%,99.9%
5,Weekday - 20:00 - 0:00,6.835,0.942,86.8%,98.2%,99.7%,100.0%,100.0%,100.0%
6,Weekend - 0:00 - 4:00,NaN,NaN,N/A,N/A,N/A,N/A,N/A,N/A
7,Weekend - 4:00 - 8:00,25.054,0.993,56.8%,81.2%,91.8%,96.5%,98.5%,99.3%
8,Weekend - 8:00 - 12:00,31.331,0.994,63.0%,86.2%,94.9%,98.1%,99.3%,99.7%
9,Weekend - 12:00 - 16:00,39.767,0.995,64.9%,87.6%,95.6%,98.5%,99.5%,99.8%


In [24]:
qos_df.to_csv("data-p3/mm1_qos_results.csv",index=False)